### Simple Wrappers

In [1]:
import inspect
from pprint import pprint
from CybORG import CybORG
from CybORG.Agents.Wrappers import *

path = str(inspect.getfile(CybORG))
path = path[:-10] + '/Shared/Scenarios/Scenario1b.yaml'

cyborg = CybORG(path,'sim')

Our demonstrations have mostly been with native CybORG, but we will now demonstrate how AI can be assisted by the use of Wrappers.

Our first Wrapper is EnumActionWrapper, which calculates all the possible actions and returns the action space as the number of such actions. This gives a relatively large number of actions for both Blue and Red team.

In [2]:
env = EnumActionWrapper(cyborg)

results = env.reset(agent='Blue')
action_space = results.action_space
print('Blue action space:',action_space)

results = env.reset(agent='Red')
action_space = results.action_space
print('Red action space:', action_space)

Blue action space: 41
Red action space: 56


We can do a similar thing for the observation space. The FlatFixedWrapper parses the internal state of CybORG and turns it into a list of floats, which can easily be converted into a vector. Unfortunately, this vector is extremely long, with length over 11,000!

In [3]:
env = FixedFlatWrapper(CybORG(path, 'sim'))

results = env.reset()
obs = results.observation
print(type(obs))
print(len(obs))

<class 'list'>
11293


The OpenAIGymWrapper converts the output of FlatFixedWrapper to a numpy array as well as conforming to other parts of the OpenAI Gym API. It requires FlatFixedWrapper and EnumActionWrapper in order to function and should always be the outermost of the provided wrappers. You must also specify an agent parameter and explitly specify the environment parameter.

In [4]:
wrappers = FixedFlatWrapper(EnumActionWrapper(cyborg))
env = OpenAIGymWrapper(env=wrappers,agent_name='Blue')

obs = env.reset()
print('Observation:',obs)
print(73*'-')
print('Action_Space:',env.action_space)
print(73*'-')
print('Observation Space:',env.observation_space)

Observation: [ 0.66666667  0.          1.         ... -1.         -1.
 -1.        ]
-------------------------------------------------------------------------
Action_Space: Discrete(41)
-------------------------------------------------------------------------
Observation Space: Box(-1.0, 1.0, (11293,), float32)


### Table Wrappers

The 'Table Wrappers' attempt to use basic logic to infer a human-friendly picture of the state by keeping track of past observations. This allows for a greatly simplified state space and much greater human readibility. However, it mercilessly exploits the current limitations of Scenario 1b and thus would have limited use on real-world cyber problems.

The first wrapper is the TrueTableWrapper, which modifies the get_agent_state method to return the true state in the form of the table.

In [6]:
env = TrueTableWrapper(cyborg)

env.reset()

true_table = env.get_agent_state('True')
print(true_table)

+----------------+-------------+-------------+-------+---------+------------+
|     Subnet     |  IP Address |   Hostname  | Known | Scanned |   Access   |
+----------------+-------------+-------------+-------+---------+------------+
| 10.0.224.64/28 | 10.0.224.66 |   Defender  | False |  False  |    None    |
| 10.0.224.64/28 | 10.0.224.67 | Enterprise0 | False |  False  |    None    |
| 10.0.224.64/28 | 10.0.224.65 | Enterprise1 | False |  False  |    None    |
| 10.0.224.64/28 | 10.0.224.68 | Enterprise2 | False |  False  |    None    |
| 10.0.199.0/28  |  10.0.199.5 |   Op_Host0  | False |  False  |    None    |
| 10.0.199.0/28  |  10.0.199.9 |   Op_Host1  | False |  False  |    None    |
| 10.0.199.0/28  |  10.0.199.3 |   Op_Host2  | False |  False  |    None    |
| 10.0.199.0/28  | 10.0.199.11 |  Op_Server0 | False |  False  |    None    |
| 10.0.169.80/28 | 10.0.169.86 |    User0    |  True |  False  | Privileged |
| 10.0.169.80/28 | 10.0.169.90 |    User1    | False |  False  |

The BlueTableWrapper provides similar functionality for the blue observation.

In [7]:
from CybORG.Agents import B_lineAgent
from CybORG.Shared.Actions import Sleep
cyborg = CybORG(path,'sim', agents={'Red':B_lineAgent})

env = BlueTableWrapper(cyborg)

results = env.reset(agent='Blue')

for i in range(3):
    results = env.step(agent='Blue')
    blue_obs = results.observation
    print(blue_obs)

+-----------------+--------------+-------------+----------+-------------+
|      Subnet     |  IP Address  |   Hostname  | Activity | Compromised |
+-----------------+--------------+-------------+----------+-------------+
| 10.0.247.224/28 | 10.0.247.237 |   Defender  |   None   |      No     |
| 10.0.247.224/28 | 10.0.247.226 | Enterprise0 |   None   |      No     |
| 10.0.247.224/28 | 10.0.247.228 | Enterprise1 |   None   |      No     |
| 10.0.247.224/28 | 10.0.247.235 | Enterprise2 |   None   |      No     |
| 10.0.196.160/28 | 10.0.196.172 |   Op_Host0  |   None   |      No     |
| 10.0.196.160/28 | 10.0.196.163 |   Op_Host1  |   None   |      No     |
| 10.0.196.160/28 | 10.0.196.170 |   Op_Host2  |   None   |      No     |
| 10.0.196.160/28 | 10.0.196.168 |  Op_Server0 |   None   |      No     |
| 10.0.192.160/28 | 10.0.192.163 |    User0    |   None   |      No     |
| 10.0.192.160/28 | 10.0.192.162 |    User1    |   None   |      No     |
| 10.0.192.160/28 | 10.0.192.172 |    

The table can also be converted into a vector. This is done by setting the output_mode parameter to 'vector'.

In [8]:
env = BlueTableWrapper(cyborg,output_mode='vector')

env.reset(agent='Blue')
for i in range(3):
    results = env.step(agent='Blue')
    blue_obs = results.observation
    print(blue_obs)
    print(76*'-')

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
----------------------------------------------------------------------------
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
----------------------------------------------------------------------------
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
----------------------------------------------------------------------------


The RedTableWrapper is the Red Team version of the BlueTableWrapper.

In [9]:
env = RedTableWrapper(cyborg)

results = env.reset(agent='Red')
print(results.observation)

for i in range(3):
    results = env.step(agent='Red')
    red_obs = results.observation
    print(red_obs)

+---------------+-------------+----------+---------+------------+
|     Subnet    |  IP Address | Hostname | Scanned |   Access   |
+---------------+-------------+----------+---------+------------+
| 10.0.32.96/28 | 10.0.32.108 |  User0   |  False  | Privileged |
+---------------+-------------+----------+---------+------------+
+---------------+-------------+-----------------+---------+------------+
|     Subnet    |  IP Address |     Hostname    | Scanned |   Access   |
+---------------+-------------+-----------------+---------+------------+
| 10.0.32.96/28 | 10.0.32.100 | UNKNOWN_HOST: 2 |  False  |    None    |
| 10.0.32.96/28 | 10.0.32.101 | UNKNOWN_HOST: 0 |  False  |    None    |
| 10.0.32.96/28 | 10.0.32.104 | UNKNOWN_HOST: 1 |  False  |    None    |
| 10.0.32.96/28 | 10.0.32.107 | UNKNOWN_HOST: 3 |  False  |    None    |
| 10.0.32.96/28 | 10.0.32.108 |      User0      |  False  | Privileged |
+---------------+-------------+-----------------+---------+------------+
+------------

### Challenge Wrapper

The challenge wrapper is three wrappers nested together: BlueTableWrapper, EnumActionWrapper and OpenAIGymWrapper.

In [18]:
env = ChallengeWrapper(env=cyborg,agent_name='Red')

obs = env.reset()

for i in range(1):
    obs, reward, done, info = env.step()
    print(obs)
    print(76*'-')
    print(reward)
    print(76*'-')
    print(done)
    print(76*'-')
    pprint(info)

[ 1  0  0  1  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
----------------------------------------------------------------------------
0.0
----------------------------------------------------------------------------
False
----------------------------------------------------------------------------
{'action': <CybORG.Shared.Actions.AbstractActions.DiscoverRemoteSystems.DiscoverRemoteSystems object at 0x7fae8dfca0d0>,
 'action_name': None,
 'action_space': 56,
 'done': False,
 'error': None,
 'error_msg': None,
 'info': None,
 'next_observation': None,
 'observation': array([ 1,  0,  0,  1,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1]),
 'parameter_mask': None,
 'reward': 0.0,
 'selection_masks': None}
